In [2]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
import pandas as pd
from datetime import datetime

In [4]:
import pandas as pd
from faker import Faker
from faker.providers import BaseProvider
from datetime import datetime
import datetime as dt 

# Inisialisasi Faker untuk lokal Indonesia
f = Faker(["id_ID"])

# --- Provider seperti struktur awal Anda ---
class NeuralProvider(BaseProvider):
    def branch(self):
        list_cabang_berbobot = [1, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 9, 10]
        return f.random_element(list_cabang_berbobot)
    def jenis_transaksi(self):
        return f.random_element(["QRis", "QRis", "Cash", "Gopay", "OVO"])

    def nomor_hp_indonesia(self):
        list_prefix = ['0812', '0813', '0857', '0878', '0896', '0821', '0852', '0858']
        prefix = f.random_element(list_prefix)
        sisa_nomor = f.numerify('########')
        return f"{prefix}{sisa_nomor}"

# --- Provider Coffee Shop dengan ID Product (Integer) ---
class CoffeeShopProvider(BaseProvider):
    # 1. Kamus Harga (Kunci Unik)
    #    Struktur ini HANYA untuk mencari harga berdasarkan ID. Tidak ada duplikasi.
    __menu_minuman = {
        101: 32500, 102: 39000, 103: 26000,  # Kopi Susu
        201: 17500, 202: 23000, 203: 40250,  # Teh
        301: 37500, 302: 31250, 303: 31250,  # Frappe
        401: 36000, 402: 42000, 403: 30000,  # Manual Brew
        501: 20250, 502: 33750, 503: 47250,  # Non-kopi
        601: 42000, 602: 63000, 603: 49000,  # Signature
    }

    # 2. Daftar Berbobot (Boleh Duplikat)
    #    Struktur ini untuk proses pemilihan acak. Ulangi ID produk sesuai bobot.
    #    Contoh: Kopi Susu (101, 102, 103) dibuat 3x lebih sering muncul.
    #            Teh (201, 202) dibuat 2x lebih sering muncul.
    __weighted_product_list = (
        # Kopi Susu (bobot 3x)
        [101, 102, 103] * 3 +
        # Teh (bobot 2x)
        [201, 202, 203, 301, 302, 303,] * 2 +
        # Produk lainnya (bobot 1x)
        [ 401, 402, 403, 501, 502, 503, 601, 602, 603]
    )

    # 3. Metode BARU untuk mengambil ID produk secara acak (berbobot)
# Tidak perlu 'import random' lagi
    def get_random_product_id(self):
        """Mengembalikan satu ID produk acak dari daftar berbobot."""
        # Gunakan generator dari Faker untuk konsistensi
        return self.generator.random_element(self.__weighted_product_list)

    # Metode ini tetap benar, digunakan untuk mengambil harga
    def get_harga(self, product_id):
        """Mengembalikan harga berdasarkan ID produk."""
        return self.__menu_minuman.get(product_id)

    # Metode ini sekarang mengembalikan daftar ID produk yang UNIK
    def get_all_product_ids(self):
        """Mengembalikan semua ID produk unik."""
        return list(self.__menu_minuman.keys())
# Menambahkan provider ke instance Faker
f.add_provider(NeuralProvider)
f.add_provider(CoffeeShopProvider)

# --- Logika Utama Generate Data ---

JUMLAH_TRANSAKSI = 1000000
headers = ['ID_Transaksi', 'No_Telephone', 'Customer_ID', 'Waktu_Transaksi',
           'ID_Cabang', 'ID_Product', 'Harga', 'Quantity', 'Total_Harga', 'Jenis_Transaksi']

data_list = []

# =============================================================================
# >>>>>>>> BAGIAN BARU: PERSIAPAN TANGGAL <<<<<<<<
# =============================================================================
# Daftar tanggal libur dan akhir pekan di Januari 2025
list_hari_libur = [1, 4, 5, 11, 12, 18, 19, 25, 26, 29] 
# Daftar tanggal hari kerja (semua tanggal 1-31 selain hari libur)
list_hari_kerja = [d for d in range(1, 32) if d not in list_hari_libur]
# =============================================================================

print("Memulai generate data transaksi...")
# Loop utama berdasarkan jumlah transaksi
for i in range(JUMLAH_TRANSAKSI):
    id_transaksi_unik = f.uuid4()
    id_cabang = f.branch()
    metode_pembayaran = f.jenis_transaksi()
    no_telp_pelanggan = f.nomor_hp_indonesia()
    customer_id_unik = f.uuid4() 

    # =============================================================================
    # >>>>>>>> LOGIKA WAKTU TRANSAKSI YANG BARU <<<<<<<<
    # =============================================================================
    # 1. Tentukan probabilitas hari: 2 dari 3 kemungkinan adalah hari libur.
    # Anda bisa atur rasionya, misal ['libur', 'libur', 'libur', 'kerja'] untuk 75%
    jenis_hari = f.random_element(['libur', 'libur', 'libur', 'kerja', 'kerja'])

    # 2. Pilih tanggal dari list yang sesuai
    if jenis_hari == 'libur':
        tanggal = f.random_element(list_hari_libur)
    else: # jika 'kerja'
        tanggal = f.random_element(list_hari_kerja)

    # 3. Generate waktu (jam, menit, detik) secara acak
    waktu_acak = f.time_object()

    # 4. Gabungkan tanggal dan waktu menjadi satu timestamp utuh
    waktu_transaksi = dt.datetime.combine(dt.date(2025, 1, tanggal), waktu_acak)
    # =============================================================================

    jumlah_item = f.random_int(1, 4)
    produk_terpilih = []
    # Loop sampai jumlah item yang unik terpenuhi
    while len(produk_terpilih) < jumlah_item:
        # Panggil metode yang sudah berbobot
        id_produk_acak = f.get_random_product_id()
        # Pastikan produk belum ada di daftar untuk transaksi ini
        if id_produk_acak not in produk_terpilih:
            produk_terpilih.append(id_produk_acak)

    for id_produk in produk_terpilih:
        harga = f.get_harga(id_produk)
        kuantitas = f.random_int(1, 3)
        total_harga = harga * kuantitas

        data_list.append({
            'ID_Transaksi': id_transaksi_unik,
            'No_Telephone': no_telp_pelanggan,
            'Customer_ID': customer_id_unik,
            'Waktu_Transaksi': waktu_transaksi, # Menggunakan waktu yang baru
            'ID_Cabang': id_cabang,
            "ID_Product": id_produk,
            "Harga": harga,
            "Quantity": kuantitas,
            "Total_Harga": total_harga,
            'Jenis_Transaksi': metode_pembayaran
        })
    
    # Progress bar sederhana
    if (i + 1) % 50000 == 0:
        print(f"Transaksi ke-{i+1}/{JUMLAH_TRANSAKSI} selesai diproses...")

# --- Menyimpan ke CSV ---
print("Menyimpan data ke file CSV...")
df = pd.DataFrame(data_list)
nama_file = 'Transaksi.csv'
df.to_csv(nama_file, index=False, encoding='utf-8')
print(f"✅ File '{nama_file}' dengan {len(df)} baris berhasil dibuat.")

Memulai generate data transaksi...
Transaksi ke-50000/1000000 selesai diproses...
Transaksi ke-100000/1000000 selesai diproses...
Transaksi ke-150000/1000000 selesai diproses...
Transaksi ke-200000/1000000 selesai diproses...
Transaksi ke-250000/1000000 selesai diproses...
Transaksi ke-300000/1000000 selesai diproses...
Transaksi ke-350000/1000000 selesai diproses...
Transaksi ke-400000/1000000 selesai diproses...
Transaksi ke-450000/1000000 selesai diproses...
Transaksi ke-500000/1000000 selesai diproses...
Transaksi ke-550000/1000000 selesai diproses...
Transaksi ke-600000/1000000 selesai diproses...
Transaksi ke-650000/1000000 selesai diproses...
Transaksi ke-700000/1000000 selesai diproses...
Transaksi ke-750000/1000000 selesai diproses...
Transaksi ke-800000/1000000 selesai diproses...
Transaksi ke-850000/1000000 selesai diproses...
Transaksi ke-900000/1000000 selesai diproses...
Transaksi ke-950000/1000000 selesai diproses...
Transaksi ke-1000000/1000000 selesai diproses...
Menyi

In [39]:
cabang = {
    'ID_Cabang': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Cabang': ['Bandung', 'Cirebon', 'Depok', 'Jakarta', 'Sumedang', 'Ciamis', 'Tangerang', 'Bogor', 'Bekasi', 'Karawang'],
}
df_cabang = pd.DataFrame(cabang)

print(df_cabang)

nama_file = 'Cabang.csv'

df_cabang.to_csv(nama_file, index=False)

   ID_Cabang     Cabang
0          1    Jakarta
1          2    Bandung
2          3      Depok
3          4    Cirebon
4          5   Sumedang
5          6     Ciamis
6          7  Tangerang
7          8      Bogor
8          9     Bekasi
9         10   Karawang


In [4]:
kategori = {
    'ID_Kategori': [1, 2, 3, 4, 5, 6],
    'Kategori': ['Coffee', 'Tea', 'Milk', 'Other', 'Snack', 'Food'],
}
df_kategori = pd.DataFrame(kategori)

print(df_kategori)

nama_file = 'Kategori.csv'

df_kategori.to_csv(nama_file, index=False)

   ID_Kategori Kategori
0            1   Coffee
1            2      Tea
2            3     Milk
3            4    Other
4            5    Snack
5            6     Food


In [2]:
product = {
    'ID_Produk': [101, 102, 103,
                  201, 202, 203,
                  301, 302, 303,
                  401, 402, 403,
                  501, 502, 503,
                  601, 602, 603],
    'ID_Kategori': [1, 1, 1,
                    2, 2, 2,
                    3, 3, 3,
                    4, 4, 4,
                    5, 5, 5,
                    6, 6, 6],
    'Produk': ['Americano', 'Cappuccino', 'Espresso',
               'Black_Tea', 'Ocha_Tea', 'Matcha_Latte',
               'Milk_Tea', 'OatMilk', 'Soy_Milk',
               'Lemon_Squash', 'Mocktail', 'Juice',
               'Cookies', 'Croissant', 'Sandwich',
               'Fried_Rice', 'Pasta', 'Noodles'],
    'Base_Price': [25000, 30000, 20000,
                   15000, 20000, 35000,
                   30000, 25000, 25000,
                   30000, 35000, 25000,
                   15000, 25000, 35000,
                   30000, 45000, 35000],
    'Increased_By': ['30%', '30%', '30%',
                     '15%', '15%', '15%',
                     '25%', '25%', '25%',
                     '20%', '20%', '20%',
                     '35%', '35%', '35%',
                     '40%', '40%', '40%'],
    'Selling_Price': [32500, 39000, 26000,
                      17500, 23000, 40250,
                      37500, 31250, 31250,
                      36000, 42000, 30000,
                      20250, 33750, 47250,
                      42000, 63000, 49000],
}
df_product = pd.DataFrame(product)

print(df_product)

nama_file = 'Produk.csv'

df_product.to_csv(nama_file, index=False)

    ID_Produk  ID_Kategori        Produk  Base_Price Increased_By  \
0         101            1     Americano       25000          30%   
1         102            1    Cappuccino       30000          30%   
2         103            1      Espresso       20000          30%   
3         201            2     Black_Tea       15000          15%   
4         202            2      Ocha_Tea       20000          15%   
5         203            2  Matcha_Latte       35000          15%   
6         301            3      Milk_Tea       30000          25%   
7         302            3       OatMilk       25000          25%   
8         303            3      Soy_Milk       25000          25%   
9         401            4  Lemon_Squash       30000          20%   
10        402            4      Mocktail       35000          20%   
11        403            4         Juice       25000          20%   
12        501            5       Cookies       15000          35%   
13        502            5     Cro

In [1]:
%%sql
SHOW DATABASES

25/08/26 07:51:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


namespace
coffeeshop_medallion
coffeeshop_rev
coffeeshoprev_medallion
default
hospital_gold
hospital_medallion
sample
Hospital
coffeeshop
test


In [2]:
%%sql
use coffeeshop_rev

++
||
++
++

In [6]:
%%sql
create database if not exists coffeeshop_rev

++
||
++
++

In [6]:
%%sql
refresh databases

++
||
++
++

In [3]:
# Tentukan path ke file CSV Anda
path_csv = 'Transaksi.csv'

# Baca file CSV ke dalam Spark DataFrame
# Spark akan secara otomatis mendeteksi header dan skema data
df_transaksi = spark.read.csv(path_csv, header=True, inferSchema=True)

# (Opsional) Tampilkan skema dan beberapa baris data untuk memastikan berhasil
print("Skema DataFrame:")
df_transaksi.printSchema()

print("\nContoh Data:")
df_transaksi.show(5)

Skema DataFrame:
root
 |-- ID_Transaksi: string (nullable = true)
 |-- No_Telephone: long (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Waktu_Transaksi: timestamp (nullable = true)
 |-- ID_Cabang: integer (nullable = true)
 |-- ID_Product: integer (nullable = true)
 |-- Harga: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Total_Harga: integer (nullable = true)
 |-- Jenis_Transaksi: string (nullable = true)


Contoh Data:
+--------------------+------------+--------------------+--------------------+---------+----------+-----+--------+-----------+---------------+
|        ID_Transaksi|No_Telephone|         Customer_ID|     Waktu_Transaksi|ID_Cabang|ID_Product|Harga|Quantity|Total_Harga|Jenis_Transaksi|
+--------------------+------------+--------------------+--------------------+---------+----------+-----+--------+-----------+---------------+
|d4988638-3449-42b...| 85733677526|d5bd2bf1-0611-421...|2025-01-02 16:03:...|        5|       203|40250

In [4]:
# Tentukan path ke file CSV Anda
path_csv = 'Produk.csv'

# Baca file CSV ke dalam Spark DataFrame
# Spark akan secara otomatis mendeteksi header dan skema data
df_product = spark.read.csv(path_csv, header=True, inferSchema=True)

# (Opsional) Tampilkan skema dan beberapa baris data untuk memastikan berhasil
print("Skema DataFrame:")
df_product.printSchema()

print("\nContoh Data:")
df_product.show(5)

Skema DataFrame:
root
 |-- ID_Produk: integer (nullable = true)
 |-- ID_Kategori: integer (nullable = true)
 |-- Produk: string (nullable = true)
 |-- Base_Price: integer (nullable = true)
 |-- Increased_By: string (nullable = true)
 |-- Selling_Price: integer (nullable = true)


Contoh Data:
+---------+-----------+----------+----------+------------+-------------+
|ID_Produk|ID_Kategori|    Produk|Base_Price|Increased_By|Selling_Price|
+---------+-----------+----------+----------+------------+-------------+
|      101|          1| Americano|     25000|         30%|        32500|
|      102|          1|Cappuccino|     30000|         30%|        39000|
|      103|          1|  Espresso|     20000|         30%|        26000|
|      201|          2| Black_Tea|     15000|         15%|        17500|
|      202|          2|  Ocha_Tea|     20000|         15%|        23000|
+---------+-----------+----------+----------+------------+-------------+
only showing top 5 rows



In [5]:
# Baca file CSV ke dalam Spark DataFrame
# Spark akan secara otomatis mendeteksi header dan skema data
df_kategori = spark.read.csv('Kategori.csv', header=True, inferSchema=True)

# (Opsional) Tampilkan skema dan beberapa baris data untuk memastikan berhasil
print("Skema DataFrame:")
df_kategori.printSchema()

print("\nContoh Data:")
df_kategori.show(5)

df_kategori.createOrReplaceTempView("Kategori")

Skema DataFrame:
root
 |-- ID_Kategori: integer (nullable = true)
 |-- Kategori: string (nullable = true)


Contoh Data:
+-----------+--------+
|ID_Kategori|Kategori|
+-----------+--------+
|          1|  Coffee|
|          2|     Tea|
|          3|    Milk|
|          4|   Other|
|          5|   Snack|
+-----------+--------+
only showing top 5 rows



In [6]:
# Baca file CSV ke dalam Spark DataFrame
# Spark akan secara otomatis mendeteksi header dan skema data
df_cabang = spark.read.csv('Cabang.csv', header=True, inferSchema=True)

# (Opsional) Tampilkan skema dan beberapa baris data untuk memastikan berhasil
print("Skema DataFrame:")
df_cabang.printSchema()

print("\nContoh Data:")
df_cabang.show(5)

df_cabang.createOrReplaceTempView("Cabang")

Skema DataFrame:
root
 |-- ID_Cabang: integer (nullable = true)
 |-- Cabang: string (nullable = true)


Contoh Data:
+---------+--------+
|ID_Cabang|  Cabang|
+---------+--------+
|        1| Bandung|
|        2| Cirebon|
|        3|   Depok|
|        4| Jakarta|
|        5|Sumedang|
+---------+--------+
only showing top 5 rows



In [7]:
df_transaksi.createOrReplaceTempView("Transaksi")

In [8]:
df_product.createOrReplaceTempView("Produk")
df_kategori.createOrReplaceTempView("Kategori")
df_cabang.createOrReplaceTempView("Cabang")

In [71]:
%%sql
show tables;

namespace,tableName,isTemporary
coffeeshop_rev,Cabang,False
coffeeshop_rev,Kategori,False
coffeeshop_rev,Produk,False
coffeeshop_rev,Transaksi,False


In [11]:
%%sql
CREATE TABLE
  if NOT EXISTS Transaksi (
        ID_Transaksi string,
        No_Telephone string,
        Customer_ID string,
        Waktu_Transaksi timestamp,
        ID_Cabang integer,
        ID_Product integer,
        Harga integer,
        Quantity integer,
        Total_Harga double,
        Jenis_Transaksi string
)

++
||
++
++

In [18]:
%%sql
CREATE TABLE
  if NOT EXISTS Produk (
        ID_Produk integer ,
        ID_Kategori integer ,
        Produk string,
        Base_Price integer ,
        Increased_By string,
        Selling_Price integer
)

++
||
++
++

In [24]:
%%sql
CREATE TABLE
  if NOT EXISTS Kategori (
        ID_Kategori integer ,
        Kategori string
)

++
||
++
++

In [25]:
%%sql
CREATE TABLE
  if NOT EXISTS Cabang (
        ID_Cabang integer ,
        Cabang string
)

++
||
++
++

In [29]:
%%sql
show tables;

namespace,tableName,isTemporary
coffeeshop_rev,Cabang,False
coffeeshop_rev,Kategori,False
coffeeshop_rev,Produk,False
coffeeshop_rev,Transaksi,False


# Medallion architecture (Bronze)

In [9]:
import pyspark
from pyspark.sql import functions as F

In [10]:
df_transaksi = spark.table("Transaksi")

In [11]:
df_product = spark.table("Produk")

In [12]:
df_kategori = spark.table("Kategori")

In [13]:
df_cabang = spark.table("Cabang")

In [15]:
df_transaksi.show(5)

+--------------------+------------+--------------------+--------------------+---------+----------+-----+--------+-----------+---------------+
|        ID_Transaksi|No_Telephone|         Customer_ID|     Waktu_Transaksi|ID_Cabang|ID_Product|Harga|Quantity|Total_Harga|Jenis_Transaksi|
+--------------------+------------+--------------------+--------------------+---------+----------+-----+--------+-----------+---------------+
|d4988638-3449-42b...| 85733677526|d5bd2bf1-0611-421...|2025-01-02 16:03:...|        5|       203|40250|       1|      40250|           QRis|
|d4988638-3449-42b...| 85733677526|d5bd2bf1-0611-421...|2025-01-02 16:03:...|        5|       603|49000|       3|     147000|           QRis|
|2c2ddc21-a957-425...| 85234785534|ad9af801-c7f9-4be...|2025-01-28 13:03:...|        9|       302|31250|       1|      31250|           Cash|
|2c2ddc21-a957-425...| 85234785534|ad9af801-c7f9-4be...|2025-01-28 13:03:...|        9|       303|31250|       3|      93750|           Cash|
|2c2dd

In [21]:
%%sql
CREATE TABLE
  if NOT EXISTS coffeeshoprev_medallion.transaksi_bronze (
    ID_Transaksi string,
    No_Telephone string,
    Customer_ID string,
    Waktu_Transaksi TIMESTAMP,
    ID_Cabang INTEGER,
    ID_Product INTEGER,
    Harga INTEGER,
    Quantity INTEGER,
    Total_Harga INTEGER,
    Jenis_Transaksi string
  )

++
||
++
++

In [52]:
%%sql
CREATE TABLE
  if NOT EXISTS coffeeshoprev_medallion.produk_bronze (
        ID_Produk integer ,
        ID_Kategori integer ,
        Produk string,
        Base_Price integer ,
        Increased_By string,
        Selling_Price integer
)

++
||
++
++

In [53]:
%%sql
CREATE TABLE
  if NOT EXISTS coffeeshoprev_medallion.kategori_bronze (
        ID_Kategori integer ,
        Kategori string
)

++
||
++
++

In [15]:
%%sql
CREATE TABLE
  if NOT EXISTS coffeeshoprev_medallion.cabang_bronze (
        ID_Cabang integer ,
        Cabang string
)

++
||
++
++

In [16]:
brz_transaksi = df_transaksi

In [17]:
brz_produk = df_product
brz_kategori = df_kategori
brz_cabang = df_cabang

In [24]:
brz_transaksi.write.insertInto("coffeeshoprev_medallion.transaksi_bronze")

In [18]:
brz_cabang.write.insertInto("coffeeshoprev_medallion.cabang_bronze")

In [20]:
brz_produk.write.insertInto("coffeeshoprev_medallion.produk_bronze")
brz_kategori.write.insertInto("coffeeshoprev_medallion.kategori_bronze")


In [25]:
%%sql

select * from coffeeshoprev_medallion.transaksi_bronze limit 10

ID_Transaksi,No_Telephone,Customer_ID,Waktu_Transaksi,ID_Cabang,ID_Product,Harga,Quantity,Total_Harga,Jenis_Transaksi
d4988638-3449-42be-8cae-f2adccc37954,85733677526,d5bd2bf1-0611-4211-8124-e767501b2293,2025-01-02 16:03:31.078623,5,203,40250,1,40250,QRis
d4988638-3449-42be-8cae-f2adccc37954,85733677526,d5bd2bf1-0611-4211-8124-e767501b2293,2025-01-02 16:03:31.078623,5,603,49000,3,147000,QRis
2c2ddc21-a957-4255-97b7-0955a3f80885,85234785534,ad9af801-c7f9-4bea-b640-01e9c5badac7,2025-01-28 13:03:13.529545,9,302,31250,1,31250,Cash
2c2ddc21-a957-4255-97b7-0955a3f80885,85234785534,ad9af801-c7f9-4bea-b640-01e9c5badac7,2025-01-28 13:03:13.529545,9,303,31250,3,93750,Cash
2c2ddc21-a957-4255-97b7-0955a3f80885,85234785534,ad9af801-c7f9-4bea-b640-01e9c5badac7,2025-01-28 13:03:13.529545,9,401,36000,1,36000,Cash
2c2ddc21-a957-4255-97b7-0955a3f80885,85234785534,ad9af801-c7f9-4bea-b640-01e9c5badac7,2025-01-28 13:03:13.529545,9,101,32500,3,97500,Cash
904df7f9-0c6a-4da6-91b9-2e85af6f4405,85743085420,93f62f8c-3b3c-460c-8546-cdae67f8850b,2025-01-12 20:52:14.671158,1,302,31250,1,31250,QRis
904df7f9-0c6a-4da6-91b9-2e85af6f4405,85743085420,93f62f8c-3b3c-460c-8546-cdae67f8850b,2025-01-12 20:52:14.671158,1,301,37500,2,75000,QRis
315e3f4d-5d73-4d0e-81c0-bec41dbe5c89,87840037806,1fdaa29b-48cc-4770-8cba-3d902c0fc353,2025-01-16 13:52:32.167531,8,103,26000,1,26000,QRis
315e3f4d-5d73-4d0e-81c0-bec41dbe5c89,87840037806,1fdaa29b-48cc-4770-8cba-3d902c0fc353,2025-01-16 13:52:32.167531,8,101,32500,3,97500,QRis


# Medallion architecture (Silver)

In [26]:
%%sql
CREATE TABLE
  if NOT EXISTS coffeeshoprev_medallion.transaksi_silver (
    ID_Transaksi string,
    No_Telephone string,
    Customer_ID string,
    Waktu_Transaksi TIMESTAMP,
    ID_Cabang INTEGER,
    ID_Product INTEGER,
    Harga INTEGER,
    Quantity INTEGER,
    Total_Harga INTEGER,
    Jenis_Transaksi string
  )

++
||
++
++

In [67]:
%%sql
CREATE TABLE
  if NOT EXISTS coffeeshoprev_medallion.produk_silver (
        ID_Produk integer ,
        ID_Kategori integer ,
        Produk string,
        Base_Price integer ,
        Increased_By string,
        Selling_Price integer
)

++
||
++
++

In [68]:
%%sql
CREATE TABLE
  if NOT EXISTS coffeeshoprev_medallion.kategori_silver (
        ID_Kategori integer ,
        Kategori string
)

++
||
++
++

In [20]:
%%sql
CREATE TABLE
  if NOT EXISTS coffeeshoprev_medallion.cabang_silver (
        ID_Cabang integer ,
        Cabang string
)

++
||
++
++

In [19]:
%%sql

drop table coffeeshoprev_medallion.cabang_silver

++
||
++
++

In [21]:
slv_transaksi = brz_transaksi
slv_produk = brz_produk
slv_cabang = brz_cabang
slv_kategori = brz_kategori

In [29]:
slv_transaksi.write.insertInto("coffeeshoprev_medallion.transaksi_silver")

In [23]:
slv_cabang.write.insertInto("coffeeshoprev_medallion.cabang_silver")

In [75]:
slv_produk.write.insertInto("coffeeshoprev_medallion.produk_silver")

slv_kategori.write.insertInto("coffeeshoprev_medallion.kategori_silver")

In [30]:
%%sql

select * from coffeeshoprev_medallion.transaksi_silver limit 10

ID_Transaksi,No_Telephone,Customer_ID,Waktu_Transaksi,ID_Cabang,ID_Product,Harga,Quantity,Total_Harga,Jenis_Transaksi
d4988638-3449-42be-8cae-f2adccc37954,85733677526,d5bd2bf1-0611-4211-8124-e767501b2293,2025-01-02 16:03:31.078623,5,203,40250,1,40250,QRis
d4988638-3449-42be-8cae-f2adccc37954,85733677526,d5bd2bf1-0611-4211-8124-e767501b2293,2025-01-02 16:03:31.078623,5,603,49000,3,147000,QRis
2c2ddc21-a957-4255-97b7-0955a3f80885,85234785534,ad9af801-c7f9-4bea-b640-01e9c5badac7,2025-01-28 13:03:13.529545,9,302,31250,1,31250,Cash
2c2ddc21-a957-4255-97b7-0955a3f80885,85234785534,ad9af801-c7f9-4bea-b640-01e9c5badac7,2025-01-28 13:03:13.529545,9,303,31250,3,93750,Cash
2c2ddc21-a957-4255-97b7-0955a3f80885,85234785534,ad9af801-c7f9-4bea-b640-01e9c5badac7,2025-01-28 13:03:13.529545,9,401,36000,1,36000,Cash
2c2ddc21-a957-4255-97b7-0955a3f80885,85234785534,ad9af801-c7f9-4bea-b640-01e9c5badac7,2025-01-28 13:03:13.529545,9,101,32500,3,97500,Cash
904df7f9-0c6a-4da6-91b9-2e85af6f4405,85743085420,93f62f8c-3b3c-460c-8546-cdae67f8850b,2025-01-12 20:52:14.671158,1,302,31250,1,31250,QRis
904df7f9-0c6a-4da6-91b9-2e85af6f4405,85743085420,93f62f8c-3b3c-460c-8546-cdae67f8850b,2025-01-12 20:52:14.671158,1,301,37500,2,75000,QRis
315e3f4d-5d73-4d0e-81c0-bec41dbe5c89,87840037806,1fdaa29b-48cc-4770-8cba-3d902c0fc353,2025-01-16 13:52:32.167531,8,103,26000,1,26000,QRis
315e3f4d-5d73-4d0e-81c0-bec41dbe5c89,87840037806,1fdaa29b-48cc-4770-8cba-3d902c0fc353,2025-01-16 13:52:32.167531,8,101,32500,3,97500,QRis


# Medallion architecture (Gold)

In [24]:
# Membuat DataFrame dim_kategori_gold
dim_kategori_gold_df = df_kategori.select("ID_Kategori", "Kategori")

# Membuat DataFrame dim_cabang_gold
dim_cabang_gold_df = df_cabang.select("ID_Cabang", "Cabang")

# Tampilkan contoh isinya
print("Contoh isi dim_kategori_gold:")
dim_kategori_gold_df.show(5)

print("Contoh isi dim_cabang_gold:")
dim_cabang_gold_df.show(5)

Contoh isi dim_kategori_gold:
+-----------+--------+
|ID_Kategori|Kategori|
+-----------+--------+
|          1|  Coffee|
|          2|     Tea|
|          3|    Milk|
|          4|   Other|
|          5|   Snack|
+-----------+--------+
only showing top 5 rows

Contoh isi dim_cabang_gold:
+---------+--------+
|ID_Cabang|  Cabang|
+---------+--------+
|        1| Bandung|
|        2| Cirebon|
|        3|   Depok|
|        4| Jakarta|
|        5|Sumedang|
+---------+--------+
only showing top 5 rows



In [32]:
# Import library yang diperlukan
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Diasumsikan df_transaksi dan df_product sudah ada
# =============================================================================
# BAGIAN 1: WINDOW FUNCTION (TIDAK BERUBAH)
# =============================================================================

# 1. Definisikan Window untuk menandai setiap transaksi unik
window_spec = Window.partitionBy("t.ID_Transaksi").orderBy("t.ID_Product")

# 2. Lakukan join dan tambahkan kolom 'penanda_transaksi'
df_joined_with_marker = (
    df_transaksi.alias("t")
    .join(
        df_product.alias("p"),
        F.col("t.ID_Product") == F.col("p.ID_Produk"),
        "left"
    )
    .withColumn("penanda_transaksi", F.when(F.row_number().over(window_spec) == 1, 1).otherwise(0))
)

# =============================================================================
# MODIFIKASI: DataFrame Fakta dengan Agregasi per TANGGAL (bukan Tahun, Bulan, Hari)
# =============================================================================

fakta_penjualan_agregat_harian_gold_df = (
    df_joined_with_marker
    
    # >>>>>>>> PERUBAHAN 1: Buat satu kolom Tanggal dari Timestamp <<<<<<<<
    # Menggunakan to_date untuk mengonversi timestamp menjadi date.
    .withColumn("Tanggal", F.to_date(F.col("t.Waktu_Transaksi")))
    .withColumn("Keuntungan", F.col("t.Total_Harga") - (F.col("p.Base_Price") * F.col("t.Quantity")))

    # >>>>>>>> PERUBAHAN 2: Kelompokkan berdasarkan kolom 'Tanggal' <<<<<<<<
    .groupBy(
        "Tanggal",  # Menggantikan Tahun, Bulan, dan Hari
        F.col("t.ID_Cabang"),
        F.col("p.ID_Kategori")
    )
    # Logika agregasi tetap sama
    .agg(
        F.sum("penanda_transaksi").alias("Jumlah_Transaksi_Akurat"),
        F.sum("t.Quantity").alias("Jumlah_Barang_Terjual"),
        F.sum("t.Total_Harga").alias("Total_Nilai_Penjualan"),
        F.sum("Keuntungan").alias("Total_Keuntungan")
    )
    # >>>>>>>> PERUBAHAN 3: Urutkan berdasarkan 'Tanggal' <<<<<<<<
    .orderBy("Tanggal", "ID_Cabang")
)


# Tampilkan skema dan contoh hasil DataFrame Fakta yang baru
print("Skema fakta_penjualan_agregat_harian_gold:")
fakta_penjualan_agregat_harian_gold_df.printSchema()

print("\nContoh isi fakta_penjualan_agregat_harian_gold:")
fakta_penjualan_agregat_harian_gold_df.show()

Skema fakta_penjualan_agregat_harian_gold:
root
 |-- Tanggal: date (nullable = true)
 |-- ID_Cabang: integer (nullable = true)
 |-- ID_Kategori: integer (nullable = true)
 |-- Jumlah_Transaksi_Akurat: long (nullable = true)
 |-- Jumlah_Barang_Terjual: long (nullable = true)
 |-- Total_Nilai_Penjualan: long (nullable = true)
 |-- Total_Keuntungan: long (nullable = true)


Contoh isi fakta_penjualan_agregat_harian_gold:


[Stage 28:>                                                       (0 + 16) / 17]

+----------+---------+-----------+-----------------------+---------------------+---------------------+----------------+
|   Tanggal|ID_Cabang|ID_Kategori|Jumlah_Transaksi_Akurat|Jumlah_Barang_Terjual|Total_Nilai_Penjualan|Total_Keuntungan|
+----------+---------+-----------+-----------------------+---------------------+---------------------+----------------+
|2025-01-01|        1|          6|                    239|                 4479|            230125000|        65750000|
|2025-01-01|        1|          4|                    368|                 4299|            155550000|        25925000|
|2025-01-01|        1|          5|                    308|                 4509|            150896250|        39121250|
|2025-01-01|        1|          2|                   1689|                 8407|            226574500|        30149500|
|2025-01-01|        1|          3|                   1060|                 8594|            286218750|        57243750|
|2025-01-01|        1|          1|      

In [33]:
%%sql
CREATE TABLE
    IF NOT EXISTS coffeeshoprev_medallion.fakta_penjualan_agregat_harian_gold (
      Tanggal DATE,
      ID_Cabang INTEGER,
      ID_Kategori INTEGER,
      Jumlah_Transaksi BIGINT,
      Jumlah_Barang_Terjual BIGINT,
      Total_Nilai_Penjualan BIGINT,
      Total_Keuntungan BIGINT
);

++
||
++
++

In [38]:
%%sql

show tables in coffeeshoprev_medallion

namespace,tableName,isTemporary
coffeeshoprev_medallion,Komposisi_Penjualan_Gold,False
coffeeshoprev_medallion,cabang_bronze,False
coffeeshoprev_medallion,cabang_silver,False
coffeeshoprev_medallion,fakta_penjualan_agregat_harian_gold,False
coffeeshoprev_medallion,kategori_bronze,False
coffeeshoprev_medallion,kategori_silver,False
coffeeshoprev_medallion,produk_bronze,False
coffeeshoprev_medallion,produk_silver,False
coffeeshoprev_medallion,transaksi_bronze,False
coffeeshoprev_medallion,transaksi_silver,False


In [36]:
fakta_penjualan_agregat_harian_gold_df.write.insertInto("coffeeshoprev_medallion.fakta_penjualan_agregat_harian_gold")

In [34]:
total_summary = (
    fakta_penjualan_agregat_harian_gold_df.agg(
        F.sum("Jumlah_Transaksi_Akurat").alias("total_transaksi"),
        F.sum("Jumlah_Barang_Terjual").alias("total_barang_terjual"),
        F.sum("Total_Nilai_Penjualan").alias("total_nilai_penjualan_rp"),
        F.sum("Total_Keuntungan").alias("total_keuntungan_rp")
    )
)
print("Ringkasan Total Keseluruhan:")
total_summary.show()

Ringkasan Total Keseluruhan:


[Stage 35:======>                                                 (2 + 15) / 17]

+---------------+--------------------+------------------------+-------------------+
|total_transaksi|total_barang_terjual|total_nilai_penjualan_rp|total_keuntungan_rp|
+---------------+--------------------+------------------------+-------------------+
|        1000000|             5002081|            170071655000|        36363445000|
+---------------+--------------------+------------------------+-------------------+



In [35]:
komposisi_kategori = (
    fakta_penjualan_agregat_harian_gold_df
    .join(
        dim_kategori_gold_df,
        fakta_penjualan_agregat_harian_gold_df.ID_Kategori == dim_kategori_gold_df.ID_Kategori,
        "inner"
    )
    .groupBy("Kategori")
    .agg(
        F.sum("Jumlah_Barang_Terjual").alias("jumlah_produk_terjual"),
        F.sum("Total_Nilai_Penjualan").alias("nilai_penjualan")
    )
    .orderBy(F.col("nilai_penjualan").desc())
)
print("Komposisi Penjualan per Kategori:")
komposisi_kategori.show()

Komposisi Penjualan per Kategori:


[Stage 45:===>                                                    (1 + 15) / 16]

+--------+---------------------+---------------+
|Kategori|jumlah_produk_terjual|nilai_penjualan|
+--------+---------------------+---------------+
|  Coffee|              1438193|    46736813500|
|    Milk|              1002100|    33401425000|
|     Tea|               999202|    26899503750|
|    Food|               520964|    26722626000|
|   Other|               519977|    18709710000|
|   Snack|               521645|    17601576750|
+--------+---------------------+---------------+



In [37]:
%%sql
CREATE TABLE
    IF NOT EXISTS coffeeshoprev_medallion.Komposisi_Penjualan_Gold (
      Kategori string,
      jumlah_produk_terjual BIGINT,
      nilai_penjualan BIGINT
);

++
||
++
++

In [38]:
komposisi_kategori.write.insertInto("coffeeshoprev_medallion.Komposisi_Penjualan_Gold")